Now that we've gotten our hands dirty with Python and some of its important operations, it's time to take on Tensorflow. Tensorflow is Google's language platform for deep learning, and it is one - if not the - most important platforms for deep learning available to the public. It can be used on its own or through higher-level frameworks such as Caffe and Keras, which can be set to use Tensorflow as a backend. 

*Note: consider adding for visual clarity https://blog.jakuba.net/2017/05/30/tensorflow-visualization.html*

### Foreword - Take a Deep Breath..

Fear not, but Tensorflow is extraordinarily complex and can be quite complicated - almost too so. It has layers upon layers of abstractions and a horde of APIs which seem to gain and fall from prominence each day. Backwards compatibility is often an issue. Such is the life of a rapidly developin ecosystem I suppose. 

Because of the above, we will come across very many methods of writing Tensorflow programs as we explore some of the most interesting models currently available. It is COMPLETELY NORMAL to feel overwhelmed and confused at first. Do not worry about this. Our method in this course is not to try and understand everything at once, but focus on areas of interest to understand how they work and how we may work with them. We will expand our knowledge in this fashion, by combining what we learn. In time, the whole picture of Tensorflow and machine/deep learning will become clearer.

### High-Level / Low-Level

One can create the same model in 30 lines of code or in hundreds of lines. We are going to do both. Why? Well, the 30-line model is so *high-level* that we won't really understand what those 30 lines of code are doing, for everything is hidden in high-level abstractions. So we can look at lower-level code, which will have many more lines but show us more clearly what exactly is happening. Once we understand how the model works on a lower-level, we can work more knowledgeably and quickly with higher-level programming again.


### Tensorflow and Python - BFFs Forever?

Tensorflow works hand-in-hand with the Python framework and many of its methods are basically exact replicas of Python/Numpy code. This is generally a good thing because it gives us familiarity as we work between the two languages. The reason for this partnership is that their similarities and compatibility support a fundamental difference in execution: **immediate/eager evaluation** vs. **graph computation**

In short, Python will evaluate (run) the code as soon as it sees it. Tensorflow, on the other hand, does not immediately run its operations. Instead, it builds a so-called *graph*, which can be considered like a system of how each variable and operation are connected to one another. In order to execute (run) the graph, we must explicitly tell Tensorflow to do so. This will become apparent in the example code below.

Final note - this introductory code has been taken from several sources, who often do a better job explaining then I can, so we will follow along with some external sites as we go. To start with, please open [this tutorial](https://jacobbuckman.com/post/tensorflow-the-confusing-parts-1/#understanding-tensorflow) to read along with the example code below.

In [1]:
import tensorflow as tf

# create a tf.Variable
one_node = tf.Variable(2)

#create a Python variable
py_var = 2

print("tf graph element: %s" % (one_node,),"python variable execution: %s" % (py_var,))

/Users/jonathansherman/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


tf graph element: <tf.Variable 'Variable:0' shape=() dtype=int32_ref> python variable execution: 2


Note that the Tensorflow code above did not evaluate, but returned instead a graph element called a **tensor** (more on that later). On the other hand, the Python variable was immediately evaluated and returned 2.

This creates some very important differences. For example, in Python if I assign 2 to the same variable, we still have 1 variable with a value of 2. If I assign this variable a value of 3, then we have the same 1 variable but with a value of 3. In Tensorflow, the graph does something quite different. Let's see for ourselves below.

In [2]:
two_node = tf.constant(2)
print(two_node)

Tensor("Const:0", shape=(), dtype=int32)


In [3]:
two_node = tf.constant(2)
two_node = tf.constant(2)
two_node = tf.constant(2)
tf.constant(3)

<tf.Tensor 'Const_4:0' shape=() dtype=int32>

Notice that in the first code block we have 'Const' and then in the second code block we have 'Const_4'. This means each time we call tf.constant it adds a tf.constant to the graph regardless of where we assign the variable in our code.

If we don't wish for this to happen, then we should create "pointer" variables instead of calling new instances of tf variables.

In [4]:
pointer_at_two_node = two_node
print(pointer_at_two_node)

Tensor("Const_3:0", shape=(), dtype=int32)


Above, we can notice 2 things. First, we didn't create another tf.constant but simply pointed the variable already storing it. Also, the tf.constant is 'Const_3' instead of 'Const_4' because we created 'Const_4' without assigning it to a Python variable. The variable we are "pointing" to contains 'Const_3'

So what we have created so far are called **nodes** in tensorflow - that is, individual values. When we want to compute them together somehow, this is called an **operation**.

In [5]:
two_node = tf.constant(2)
three_node = tf.constant(3)
sum_node = two_node + three_node ## equivalent to tf.add(two_node, three_node)
print(sum_node)

Tensor("add:0", shape=(), dtype=int32)


Though very simple, we have our 1st **computational graph**. Notice that it has not been evaluated yet, for when we print sum_node we get the operation "add", not 5 like we would if this were just in Python.

In order to evaluate a graph, we need to begin what is called a **session**. This means the graph is interactive and can be executed (evaluated)

In [6]:
sess = tf.Session()

print(sess.run(sum_node))

5


Perhaps we want the values of nodes as well as operations

In [7]:
print(sess.run([two_node, sum_node]))

[2, 5]


And we can return them to Python variables if we want

In [8]:
node_val, op_result = sess.run([two_node, sum_node])
print(node_val, op_result, node_val+op_result)

2 5 7


Sometimes when creating a graph, we want to add in values at a later point (for instance, perhaps we need to insert an image file into our graph and evaluate it). To do so, we can make use of **tf.placeholder** and **feed_dict**. As the name suggests, placeholder simply holds an empty location in the graph. It has no value until we pass in a value using feed_dict. 

Below we'll feed a tf.placeholder with a Python variable of value 2.

In [9]:
input_placeholder = tf.placeholder(tf.int32)
feed_value = 2
print(sess.run(input_placeholder, feed_dict={input_placeholder: feed_value}))

2


So far we've evaluated graphs that contain static or "no-ancestor" nodes - they don't update. However, with machine/deep learning, **we often need nodes that update** - this is how they learn! Most of the **parameters** we want to train/learn will be implemented as Tensorflow **variables**. 

Also, we've only been dealing with **singular values**, called **scalars** so far. Generally, we will be working with **matrices (and "tensors")** of varying dimensions. To do this, we assign variables a **shape**, which is their dimensions. 

Finally, we can also give the variable a **name** to make it easier to identify, or Tensorflow will do it for us (e.g. 'const_5', etc.)

In [10]:
count_variable = tf.get_variable("count", [2, 3])
zero_node = tf.constant([[1, 2, 3], [4, 5, 6]])
assign_node = tf.assign(count_variable, zero_node)
sess = tf.Session()
sess.run(assign_node)
print(sess.run(count_variable))

TypeError: Input 'value' of 'Assign' Op has type int32 that does not match type float32 of argument 'ref'.

Note we received an error exception for data type incompatibility. We often need to declare which data type we want to work with to avoid this error

In [11]:
count_variable = tf.get_variable("count", [2, 3], dtype=tf.float32)
zero_node = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
assign_node = tf.assign(count_variable, zero_node)
sess = tf.Session()
sess.run(assign_node)
print(sess.run(count_variable))

ValueError: Variable count already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-10-5dd14a5ec98a>", line 1, in <module>
    count_variable = tf.get_variable("count", [2, 3])
  File "/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


Oh, now another error exception - this time for variable reuse. This is because we have given it a specific name, so Tensorflow cannot simply add another variable like it did when we allowed it to autoname variables. And because the error in our code was further below the count_variable, it thinks we are trying to add it again and it is blocking us. Simple workaround when this happens - comment out the count_variable until the block of code works, then include it next time

In [12]:
#count_variable = tf.get_variable("count", [2, 3], dtype=tf.float32)
zero_node = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
assign_node = tf.assign(count_variable, zero_node)
sess = tf.Session()
sess.run(assign_node)
print(sess.run(count_variable))

[[1. 2. 3.]
 [4. 5. 6.]]


Or reset the "default graph" which will clear variables and let us start anew.

In [13]:
tf.reset_default_graph()
count_variable = tf.get_variable("count", [2, 3], dtype=tf.float32)
zero_node = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
assign_node = tf.assign(count_variable, zero_node)
sess = tf.Session()
sess.run(assign_node)
print(sess.run(count_variable))

[[1. 2. 3.]
 [4. 5. 6.]]


Great, so it finally worked, but perhaps is still confusing. Why this "get_variable" and "assign" process? Why can't we just create a variable in one step. Well, we can with **tf.Variable()**, but the "pointer" method above makes sure we aren't creating unnecessary graph elements like we've shown before. For instance, if there already is a variable with the name we are looking for, we don't want to create another. tf.Variable() will always try to create another.

In [14]:
pointer_count_variable = tf.get_variable("count", [2, 3], dtype=tf.float32)

ValueError: Variable count already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-13-63a0e12acad5>", line 2, in <module>
    count_variable = tf.get_variable("count", [2, 3], dtype=tf.float32)
  File "/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


In [15]:
create_additional_count_var = tf.Variable(1, name = 'count')
print(create_additional_count_var)

<tf.Variable 'count_1:0' shape=() dtype=int32_ref>


See above, get_variable was an error exception as we expected, whereas tf.Variable created a new node with a 0 value, autonamed 'count_1'. If this is not what we wanted, we will have problems with our graph.

Finally, there is another way to create and evaluate variables other than using the 'assign' method. We can use "initializers", which will set all variables or constants to an initial value. Since these will generally be updated by our training, we often don't care what the initial value is, or just set it to 0.

In [16]:
const_init_node = tf.constant_initializer(0.)
count_variable = tf.get_variable("count", [], initializer=const_init_node)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
print(sess.run(count_variable))

ValueError: Variable count already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-13-63a0e12acad5>", line 2, in <module>
    count_variable = tf.get_variable("count", [2, 3], dtype=tf.float32)
  File "/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2881, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/Users/jonathansherman/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2821, in run_ast_nodes
    if self.run_code(code, result):


Oh yeah, that annoying error again. We've "accidentally" created a bunch of nodes we don't want, and are trying to change an existing node. Let's reset all of the variables and get this right by using **tf.reset_default_graph**

In [17]:
tf.reset_default_graph()

const_init_node = tf.constant_initializer(0.)
count_variable = tf.get_variable("count", [], initializer=const_init_node)
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
print(sess.run(count_variable))

0.0


Hopefully this gives you an idea of how Tensorflow works with but is different than Python, and how the concept of building a graph with nodes and operations differes and *then* evaluating it is different than the immediate evaluation we are probably used to with Python or other coding languages.

There is plenty more to learn about Tensorflow, but this is a good start. 

Let's see how this is put together in a toy **linear regression** example.

Most importantly, pay attention to how the variables are updated as the model trains and evaluates. This is called **optimization**, and a typical optimization will do something along the likes of the following:

1. Get an input and true_output
2. Compute a “guess” based on the input and your parameters
3. Compute a “loss” based on the difference between your guess and the true_output
4. Update the parameters according to the gradient of the loss

In [18]:
### build the graph
## first set up the parameters
m = tf.get_variable("m", [], initializer=tf.constant_initializer(0.))
b = tf.get_variable("b", [], initializer=tf.constant_initializer(0.))
init = tf.global_variables_initializer()

## then set up the computations
input_placeholder = tf.placeholder(tf.float32)
output_placeholder = tf.placeholder(tf.float32)

x = input_placeholder
y = output_placeholder
y_guess = m * x + b

loss = tf.square(y - y_guess)

## A "Hyperparameter" - you control this!
learning_rate = 0.001

## finally, set up the optimizer and minimization node
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss)

### start the session
sess = tf.Session()
sess.run(init)

### perform the training loop
import random

## set up problem
true_m = random.random()
true_b = random.random()

for update_i in range(10000):
    ## (1) get the input and output
    input_data = random.random()
    output_data = true_m * input_data + true_b
    
    ## (2), (3), and (4) all take place within a single call to sess.run()!
    _loss, _ = sess.run([loss, train_op], feed_dict={input_placeholder: input_data, output_placeholder: output_data})
    print(update_i, _loss)
    

### finally, print out the values we learned for our two variables
print("True parameters:     m=%.4f, b=%.4f" % (true_m, true_b))
print("Learned parameters:  m=%.4f, b=%.4f" % tuple(sess.run([m, b])))

0 1.404838
1 1.5526367
2 1.2415051
3 1.0039726
4 1.5366151
5 1.0367047
6 1.5597478
7 1.1151637
8 1.0099484
9 1.5500116
10 1.1803026
11 1.4646165
12 0.9403242
13 1.1677575
14 1.0229205
15 1.4484422
16 1.2726054
17 0.9284853
18 0.9572506
19 1.3446517
20 1.4289455
21 1.1109977
22 1.1928486
23 0.9247476
24 1.0606116
25 1.3345537
26 0.9346176
27 1.0096587
28 1.1067114
29 0.8900814
30 0.97867066
31 1.1784296
32 1.0720402
33 1.3013961
34 1.0719833
35 1.061161
36 1.0804956
37 1.0742731
38 1.2072421
39 0.9438178
40 1.3017024
41 0.99103975
42 1.0738442
43 1.2137102
44 0.8575057
45 1.0548229
46 1.2748735
47 1.0645579
48 0.8502737
49 0.8854814
50 0.92836344
51 1.1048368
52 1.1127837
53 1.0152959
54 1.1296924
55 0.7760136
56 0.99739295
57 1.0432847
58 0.8813136
59 1.129644
60 0.89495677
61 0.91611534
62 1.0397027
63 0.96215063
64 0.77486163
65 1.0865383
66 0.9536153
67 0.8436274
68 1.0390111
69 0.9572837
70 1.0336936
71 0.7320428
72 0.75937974
73 1.0886071
74 0.76389146
75 0.86146295
76 0.8023171
7

569 0.06779515
570 0.07942614
571 0.06376904
572 0.06424236
573 0.06947399
574 0.07541414
575 0.074794665
576 0.07238726
577 0.07535929
578 0.07117371
579 0.085201286
580 0.08042979
581 0.07449181
582 0.07064096
583 0.083042696
584 0.07782719
585 0.05919818
586 0.077851266
587 0.06517817
588 0.066062585
589 0.06897272
590 0.06300719
591 0.0679047
592 0.06474293
593 0.07234012
594 0.07478283
595 0.07217151
596 0.05442654
597 0.053180262
598 0.062954806
599 0.06529381
600 0.05391093
601 0.067268915
602 0.056356266
603 0.0744923
604 0.0659414
605 0.072141826
606 0.07600295
607 0.0743469
608 0.069069155
609 0.06489742
610 0.07247591
611 0.062449556
612 0.06647488
613 0.048702188
614 0.053949907
615 0.06240187
616 0.05151198
617 0.051268116
618 0.058389388
619 0.06925327
620 0.049623594
621 0.06518347
622 0.05289003
623 0.05523655
624 0.050905958
625 0.056875795
626 0.053539455
627 0.0540544
628 0.06424291
629 0.042629767
630 0.047090556
631 0.05993759
632 0.062391717
633 0.058093842
634 0.

1090 0.00064728345
1091 0.012187954
1092 0.0072358325
1093 0.0013634174
1094 0.005688683
1095 0.008202078
1096 0.005554757
1097 0.008129064
1098 0.0012426429
1099 0.0008153288
1100 0.00058717973
1101 0.011129391
1102 0.0015437846
1103 0.008736704
1104 0.003038358
1105 0.0007922862
1106 0.004842167
1107 0.0035654295
1108 0.0075696036
1109 0.00023580137
1110 0.00077346776
1111 0.015078623
1112 0.0052094925
1113 0.014362246
1114 0.0036218127
1115 0.0005967859
1116 0.007267058
1117 0.010978328
1118 0.0025690345
1119 0.0034172786
1120 0.013246288
1121 0.012798552
1122 0.007454628
1123 0.0031229686
1124 0.00622611
1125 0.015198308
1126 0.00049074047
1127 0.0008704263
1128 0.000445398
1129 0.0031632015
1130 0.017170647
1131 0.013789476
1132 0.01655357
1133 0.010215323
1134 0.0030155734
1135 0.0079937335
1136 9.766518e-05
1137 0.0053828917
1138 0.00011341317
1139 0.008443494
1140 0.0007293128
1141 0.0013725265
1142 0.009800877
1143 0.0041962164
1144 0.010669789
1145 0.01061025
1146 0.009919
11

1607 0.0004509855
1608 0.0004056697
1609 9.4415213e-07
1610 7.7869125e-05
1611 0.0013624667
1612 0.00011243521
1613 0.00036450307
1614 0.006091945
1615 0.0018347042
1616 0.0014984547
1617 0.00015139594
1618 0.0077583995
1619 0.00022810728
1620 8.616867e-05
1621 0.00032083227
1622 0.0010184727
1623 0.00038998714
1624 0.000288873
1625 0.0005973568
1626 0.00012442359
1627 1.4018475e-05
1628 0.006074335
1629 0.0001902532
1630 0.0013078037
1631 0.003158309
1632 0.0018409902
1633 0.006238071
1634 0.0011124047
1635 0.0042258753
1636 0.00251456
1637 0.00040250656
1638 0.0048583313
1639 0.004043103
1640 0.0001985399
1641 1.9669988e-06
1642 0.0008733197
1643 3.0779304e-06
1644 6.314826e-08
1645 0.0018868665
1646 0.007131894
1647 0.0036942144
1648 0.0029442527
1649 0.0012998576
1650 0.00024157694
1651 0.007228685
1652 0.0040015583
1653 8.597844e-05
1654 0.0017499437
1655 0.00091616344
1656 0.0003590342
1657 0.001420836
1658 0.00025172747
1659 0.0020062963
1660 0.0015715241
1661 8.055971e-05
1662 

2161 0.0029144187
2162 0.002142536
2163 0.0028661478
2164 0.0046557765
2165 0.00044929096
2166 4.8554983e-05
2167 0.001498704
2168 0.00079561153
2169 0.0029855552
2170 2.0180605e-05
2171 0.0022446583
2172 0.0034397615
2173 0.00017981137
2174 9.812517e-05
2175 0.0030052324
2176 0.00059128297
2177 0.00013537197
2178 0.0012179399
2179 0.0013603643
2180 0.00039362567
2181 7.0936396e-05
2182 0.0036226306
2183 0.00078618433
2184 6.8347974e-05
2185 0.0029191507
2186 0.0019923097
2187 0.0012877401
2188 0.004692615
2189 0.00048718182
2190 0.0036306065
2191 0.0020952139
2192 0.0001495826
2193 0.0018641821
2194 0.0012443413
2195 0.0019704052
2196 0.0045579798
2197 0.00078483456
2198 0.004804241
2199 0.0014383422
2200 0.0011453433
2201 0.0028164638
2202 0.0027218496
2203 0.004110946
2204 0.00012688231
2205 0.0014647113
2206 0.0038543292
2207 6.937479e-05
2208 0.0019826372
2209 0.000905507
2210 0.0005582935
2211 0.0015875377
2212 7.2859825e-06
2213 0.0005971762
2214 0.0006957693
2215 0.0045171464
2

...

7550 0.00013711685
7551 5.058012e-06
7552 0.0007559175
7553 0.00068691775
7554 0.00025102438
7555 0.0004132924
7556 0.00011094106
7557 1.3981008e-05
7558 0.00040589063
7559 0.00050660584
7560 1.27607955e-05
7561 4.0565614e-05
7562 0.00056478503
7563 0.0004943597
7564 0.0008866656
7565 0.0008769944
7566 7.8366435e-07
7567 0.00106991
7568 0.00039283614
7569 0.0005974734
7570 0.0006574285
7571 5.1603416e-07
7572 1.5938613e-05
7573 0.00058118097
7574 2.3030481e-05
7575 0.0002648772
7576 0.00036181335
7577 0.0007633264
7578 0.00010800255
7579 0.00073267764
7580 0.0009057653
7581 0.0004568779
7582 0.0005451613
7583 0.00027966793
7584 0.00044690882
7585 0.00019932342
7586 0.00047332066
7587 0.0006946189
7588 0.0005387451
7589 0.00041902185
7590 0.00085903
7591 1.9217723e-05
7592 0.0012024922
7593 0.0007556422
7594 0.000114018265
7595 0.00029141933
7596 0.0010801349
7597 3.9313538e-05
7598 0.0003506047
7599 1.07892065e-05
7600 1.2716547e-05
7601 0.00071401824
7602 4.23307e-05
7603 0.0010051064

8077 0.00011861822
8078 7.530491e-05
8079 2.0520474e-05
8080 0.00084321306
8081 0.00016024006
8082 0.0010334826
8083 1.9089171e-08
8084 0.00044719616
8085 7.815552e-07
8086 1.8474907e-06
8087 0.00039547737
8088 8.640786e-05
8089 0.00065937394
8090 0.0007069515
8091 2.6312546e-07
8092 0.00047982665
8093 0.0003269337
8094 8.096251e-05
8095 2.1818903e-06
8096 0.0006357857
8097 0.00016359234
8098 3.651634e-06
8099 7.4816317e-06
8100 0.00035645926
8101 0.00016920803
8102 0.00049521885
8103 4.980225e-05
8104 0.00024110655
8105 0.0001561469
8106 8.302511e-06
8107 0.00037242955
8108 0.00034035053
8109 0.00020751468
8110 1.1602786e-05
8111 0.00030491364
8112 0.00082894973
8113 6.7364033e-07
8114 0.00016104992
8115 0.0005851594
8116 8.3293244e-08
8117 0.0005459409
8118 0.00017166071
8119 0.00019020386
8120 0.0005957845
8121 0.00021208687
8122 0.0006339595
8123 0.00063878304
8124 0.00049196655
8125 0.00060856796
8126 0.00017440172
8127 4.606906e-05
8128 9.497203e-05
8129 4.0342697e-05
8130 3.8366

8598 0.0008657514
8599 6.689911e-05
8600 0.0008180405
8601 0.00072867225
8602 0.0003958805
8603 0.00011368755
8604 0.0007362346
8605 0.00077417743
8606 0.00020328794
8607 8.053656e-06
8608 0.00021088721
8609 0.00015392045
8610 0.00029762217
8611 0.0005486684
8612 0.0006144874
8613 0.00040149313
8614 9.970915e-05
8615 3.067458e-05
8616 1.4436273e-06
8617 0.00016337591
8618 0.00027675298
8619 3.3720353e-05
8620 0.00078661554
8621 0.00010140902
8622 0.00065912295
8623 7.845722e-05
8624 0.0004435332
8625 1.0158114e-05
8626 5.3693086e-05
8627 0.000843535
8628 0.00059128297
8629 1.47655155e-05
8630 0.00047302505
8631 6.950194e-05
8632 2.0740305e-05
8633 0.00012796152
8634 0.00026676248
8635 0.00037562495
8636 0.00013704985
8637 1.8984316e-05
8638 1.0638591e-05
8639 0.00037373282
8640 0.00038070642
8641 0.0006922834
8642 6.44665e-05
8643 0.0004104763
8644 0.00021115737
8645 5.094103e-05
8646 0.0004192122
8647 5.944379e-05
8648 3.6903028e-05
8649 0.0006842054
8650 0.0001294329
8651 0.000251035

9128 8.349193e-07
9129 0.00049168104
9130 6.468108e-05
9131 0.0004351726
9132 0.00047814124
9133 0.00057454983
9134 0.00025721555
9135 9.406591e-06
9136 0.0002581379
9137 0.00025913864
9138 0.0004506919
9139 7.690448e-06
9140 0.00052471785
9141 0.00051509304
9142 0.00023683494
9143 3.5998803e-06
9144 3.267742e-06
9145 8.954407e-06
9146 2.3938817e-06
9147 5.881496e-05
9148 0.0006000754
9149 7.760846e-09
9150 0.00057951245
9151 0.00021573428
9152 5.8004112e-05
9153 2.6621643e-05
9154 4.5687946e-05
9155 0.0005658933
9156 0.00010161801
9157 0.00037075664
9158 0.00017510772
9159 5.1230723e-05
9160 5.2348705e-06
9161 0.0005513887
9162 0.00035649078
9163 0.0003548451
9164 0.000102634775
9165 0.00013762263
9166 0.00046570587
9167 8.5839224e-07
9168 0.00016153135
9169 0.000191528
9170 4.8062822e-05
9171 0.00019176563
9172 4.2271773e-07
9173 0.00056233996
9174 9.1831535e-05
9175 6.505937e-05
9176 4.2615297e-06
9177 0.0006486156
9178 0.00021070376
9179 0.00016509918
9180 0.0005349114
9181 5.32251

9772 2.0535317e-06
9773 0.00026991815
9774 0.000121489116
9775 0.00031100961
9776 0.0001377905
9777 0.00016861928
9778 8.292431e-05
9779 0.00037090355
9780 2.542949e-06
9781 0.00016873695
9782 8.691168e-05
9783 0.00019311506
9784 0.00023940291
9785 0.0004354064
9786 0.0002860676
9787 0.00022613109
9788 0.00025933827
9789 0.00012964183
9790 0.00021266712
9791 0.00012510801
9792 0.00036776494
9793 4.573308e-05
9794 1.5014882e-06
9795 0.0005742841
9796 0.00020306364
9797 0.00038334398
9798 5.400801e-05
9799 1.2300012e-07
9800 5.5712235e-10
9801 0.00031764165
9802 8.831407e-06
9803 0.00044137676
9804 7.700892e-05
9805 0.00021629494
9806 0.00011102394
9807 4.5348543e-07
9808 0.00021686335
9809 1.8828814e-09
9810 4.5375844e-05
9811 1.6498891e-06
9812 2.3882621e-05
9813 8.156829e-06
9814 2.2264899e-06
9815 0.00031802844
9816 0.0006267375
9817 0.00034611032
9818 0.00011990966
9819 0.00013010374
9820 9.289241e-05
9821 8.4291176e-05
9822 2.9870469e-05
9823 3.1675612e-05
9824 6.447928e-06
9825 5.

Often times if we are developing a model, we want to inspect the graph and its variables and performance. We can (and usually will) just print or plot the values for quick visualization. However, Tensorflow has greater tools for inspecting our graph and model, such as Tensorboard. Below we will run the same toy linear regression model but save the variable data we are interested in. 

In [ ]:
import datetime

tf.reset_default_graph()

### build the graph
## first set up the parameters
m = tf.get_variable("m", [], initializer=tf.constant_initializer(0.))
## save our variable for Tensorboard
tf.summary.histogram("m_prediction",m)
b = tf.get_variable("b", [], initializer=tf.constant_initializer(0.))
tf.summary.histogram("b_prediction",b)

init = tf.global_variables_initializer()

## then set up the computations
input_placeholder = tf.placeholder(tf.float32, name="input")
output_placeholder = tf.placeholder(tf.float32, name="output")

x = input_placeholder
y = output_placeholder
tf.summary.histogram("y_true",output_placeholder)
y_guess = m * x + b
tf.summary.histogram("y_guess",y_guess)

loss = tf.square(y - y_guess)
tf.summary.scalar("loss", loss)

## finally, set up the optimizer and minimization node
optimizer = tf.train.GradientDescentOptimizer(1e-3)
train_op = optimizer.minimize(loss)

### start the session
sess = tf.Session()
## merge all our Tensorboard summaries and create file directory
summaryMerged = tf.summary.merge_all()
filename="./summary_log/run"+datetime.datetime.now().strftime("%Y-%m-%d--%H-%M-%s")
writer = tf.summary.FileWriter(filename, sess.graph)

## run variables initializer
sess.run(init)

### perform the training loop
import random

## set up problem
true_m = random.random()
true_b = random.random()

for update_i in range(20000):
    ## (1) get the input and output
    input_data = random.random()
    output_data = true_m * input_data + true_b
    
    ## (2), (3), and (4) all take place within a single call to sess.run()!
    _loss, _, sumOut = sess.run([loss, train_op,summaryMerged], feed_dict={input_placeholder: input_data, output_placeholder: output_data})
    print(update_i, _loss)
    
    if update_i % 100 == 0:
        ## write all of the variables to file
        writer.add_summary(sumOut, update_i)

### finally, print out the values we learned for our two variables
print("True parameters:     m=%.4f, b=%.4f" % (true_m, true_b))
print("Learned parameters:  m=%.4f, b=%.4f" % tuple(sess.run([m, b])))

Now, open up your terminal console and cd to the same file directory location our program is in

Then run ```tensorboard --logdir=./summary_log/``` and copy the location http address into a new browser window. You should see our graph, scalars, histogram and even more. Run the model more than once, and you can compare the results between runs. Very useful!

For more details see the source I adapted this from: https://thecodacus.com/tensorboard-tutorial-visualize-networks-graphically/

We can also just visualize our graph inline with Jupyter via [the following code](https://stackoverflow.com/questions/38189119/simple-way-to-visualize-a-tensorflow-graph-in-jupyter):

In [ ]:
import numpy as np
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
show_graph(tf.get_default_graph().as_graph_def())

You can zoom and click each node and operation to see its inputs and outputs, which can be very helpful for debugging or just understanding what is going on.

Furthermore and importantly, we can save and reload trained models. This is essential for any serious machine/deep learning project that takes significant time to train. Now, we can train something for weeks or even months, or even forever.

In [19]:
tf.reset_default_graph()

### build the graph
## first set up the parameters
m = tf.get_variable("m", [], initializer=tf.constant_initializer(0.))
## save our variable for Tensorboard
tf.summary.histogram("m_prediction",m)
b = tf.get_variable("b", [], initializer=tf.constant_initializer(0.))
tf.summary.histogram("b_prediction",b)

init = tf.global_variables_initializer()

## then set up the computations
input_placeholder = tf.placeholder(tf.float32)
output_placeholder = tf.placeholder(tf.float32)

x = input_placeholder
y = output_placeholder
tf.summary.histogram("y_true",output_placeholder)
y_guess = m * x + b
tf.summary.histogram("y_guess",y_guess)

loss = tf.square(y - y_guess)
tf.summary.scalar("loss", loss)

# a "hyperparameter"
learning_rate = 0.001

## finally, set up the optimizer and minimization node
optimizer = tf.train.GradientDescentOptimizer(1e-3)
train_op = optimizer.minimize(loss)

### start the session
sess = tf.Session()
## merge all our Tensorboard summaries and create file directory
summaryMerged = tf.summary.merge_all()
filename="./summary_log/run"+datetime.datetime.now().strftime("%Y-%m-%d--%H-%M-%s")
writer = tf.summary.FileWriter(filename, sess.graph)

## create tf.saver
saver = tf.train.Saver()

## run variables initializer
sess.run(init)

### perform the training loop
import random

## set up problem
true_m = random.random()
true_b = random.random()

for update_i in range(20000):
    ## (1) get the input and output
    input_data = random.random()
    output_data = true_m * input_data + true_b
    
    ## (2), (3), and (4) all take place within a single call to sess.run()!
    _loss, _, sumOut = sess.run([loss, train_op,summaryMerged], feed_dict={input_placeholder: input_data, output_placeholder: output_data})
    print(update_i, _loss)
    
    if update_i % 100 == 0:
        ## write all of the variables to file
        writer.add_summary(sumOut, update_i)

## save to directory
save_path = saver.save(sess, './model/weights')

### finally, print out the values we learned for our two variables
print("True parameters:     m=%.4f, b=%.4f" % (true_m, true_b))
print("Learned parameters:  m=%.4f, b=%.4f" % tuple(sess.run([m, b])))
print("Model saved in path: %s" % save_path)

NameError: name 'datetime' is not defined

So now we should have a folder "model" with our weight files saved (these are all of our variables). We can now reload the model and continue working with it, without having to retrain from scratch. More on working with and saving models here: https://www.tensorflow.org/guide/saved_model

In [ ]:
sess = tf.Session()

with tf.Session() as sess:
    # Reset all variables to initial state
    sess.run(init)
    
    # Verify reset variable
    print('reset value of var m',m.eval())
    
    reader = tf.train.Saver()
    reader.restore(sess, './model/weights')
    
    # Verify restored variable
    print('restored value of var m',m.eval())

One tricky but essential aspect of working with Tensorflow is managing graphs and sessions. Recall that the graph contains all the nodes and operations we build for our learning model. We call a session when we want to evaluate the graph.

Things can get confusing though if we:
1. have more than one graph, 
2. need to run more than one session,
3. want to reset or update graph elements
4. want to access graph elements for input or output

We won't cover all techniques here, but let's get a bit more familiar with graph and session usage before moving on.

We can assign our default graph (which is automatically instantiated by Tensorflow) to a variable if we wish (though normally we wouldn't - this is just to demonstrate. Normally, you can name a graph if you want at the outset or when constructing a 2nd, different graph).

In [ ]:
graph = tf.get_default_graph()

In [ ]:
# Verify it's the graph we want
show_graph(graph)

Another way to access all of the nodes in our graph is to call them via graph_def

In [ ]:
graphdef = graph.as_graph_def()
for node in graphdef.node:
    print(node)

We can do similarly access all of the operations in our graph

In [ ]:
for op in graph.get_operations(): 
    print(op.name)

And get a specific operation by name

In [ ]:
some_op = graph.get_operation_by_name("gradients/mul_grad/Shape")
print(some_op)

Or layer and type

In [ ]:
layers = [op.name for op in graph.get_operations() if op.type=='Const' and 'gradients' in op.name]
print(layers)

In [ ]:
#close and clear our default graph
sess.close()
tf.reset_default_graph()

In [ ]:
show_graph(tf.get_default_graph())

In [ ]:
train_graph = tf.Graph()
eval_graph = tf.Graph()
infer_graph = tf.Graph()

x = tf.constant(0)

with train_graph.as_default():
    with tf.variable_scope("share", reuse=True):
        a = tf.constant([[-1, -2, -23], [-4, -5, -6]])
        b = tf.constant([[1, 2, 3], [4, 5, 6]])
        c = tf.add(a,b)

with eval_graph.as_default():
    with tf.variable_scope("share_again", reuse=True):
        c1 = c

with infer_graph.as_default():
    with tf.variable_scope("sharing_through_2", reuse=True):
        c2 = c1

In [ ]:
show_graph(train_graph)